In [1]:
!pip install pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import tensorflow as tf
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
zip_file = drive.CreateFile({'id': '1C5FOGXLAAhvdaOuTWukGysyuj1NjIpVy'})
zip_file.GetContentFile('complete_data_reduced.zip')
validation_file = drive.CreateFile({'id':'1Xiy766wWCAo8gcNSyEFzwn2xaUHtxXqx'})
validation_file.GetContentFile('validation_reduced.zip')

In [0]:
# !unzip 'complete_data_reduced.zip'
# !unzip 'validation_reduced.zip'

In [5]:
from keras.models import Sequential
from keras.utils.np_utils import to_categorical  
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D,SeparableConv2D,Conv2D
from keras.optimizers import Adam, RMSprop
from keras.layers.pooling import MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization,Activation,Input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob,string

## file paths training data
angry_path = 'combined_data_reduced/angry/*'
closed_path = 'combined_data_reduced/closed/*'
disgust_path = 'combined_data_reduced/disgusted/*'
fear_path = 'combined_data_reduced/fearful/*'
happy_path = 'combined_data_reduced/happy/*'
neutral_path = 'combined_data_reduced/neutral/*'
sad_path = 'combined_data_reduced/sad/*'
surprised_path = 'combined_data_reduced/surprised/*'

## file path validation
angry_path_val = 'validation_reduced/angry/*'
closed_path_val = 'validation_reduced/closed/*'
disgust_path_val = 'validation_reduced/disgusted/*'
fear_path_val = 'validation_reduced/fearful/*'
happy_path_val = 'validation_reduced/happy/*'
neutral_path_val = 'validation_reduced/neutral/*'
sad_path_val = 'validation_reduced/sad/*'
surprised_path_val = 'validation_reduced/surprised/*'


#list files training
angry_files=glob.glob(angry_path)
closed_files = glob.glob(closed_path)
disgust_files = glob.glob(disgust_path)
fear_files = glob.glob(fear_path)
happy_files = glob.glob(happy_path)
neutral_files = glob.glob(neutral_path)
sad_files = glob.glob(sad_path)
surprised_path = glob.glob(surprised_path)

#list files validation
angry_files_val=glob.glob(angry_path_val)
closed_files_val = glob.glob(closed_path_val)
disgust_files_val = glob.glob(disgust_path_val)
fear_files_val = glob.glob(fear_path_val)
happy_files_val = glob.glob(happy_path_val)
neutral_files_val = glob.glob(neutral_path_val)
sad_files_val = glob.glob(sad_path_val)
surprised_path_val = glob.glob(surprised_path_val)

In [0]:
batch_size = 64
num_epoch = 60

training_directory='combined_data_reduced'
validation_directory = 'validation_reduced'

In [8]:

train_datagen = ImageDataGenerator(rescale=1./255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True
                                  )
val_datagen = ImageDataGenerator(rescale=1./255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True
                                )

train_generator = train_datagen.flow_from_directory(
        training_directory,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_directory,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 31527 images belonging to 8 classes.
Found 7591 images belonging to 8 classes.


In [0]:
def create_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    #model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(256, kernel_size=(1, 1), activation='relu'))
    model.add(Conv2D(512, kernel_size=(1, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))
 
    return model

In [0]:
# from keras.models import Model
# from keras import layers
# from keras.regularizers import l2
# def create_model():
#     model = Sequential()
#     model.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same',
#                             name='image_array', input_shape=(48,48,1)))
#     model.add(BatchNormalization())
#     model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     #model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(.25))

#     model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     #model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(.25))

#     model.add(Convolution2D(filters=64, kernel_size=(2, 2), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Convolution2D(filters=128, kernel_size=(2, 2), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(.3))

#     model.add(Convolution2D(filters=128, kernel_size=(2, 2), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(.3))

#     model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dense(512,activation='relu'))
#     model.add(Convolution2D(
#         filters=8, kernel_size=(3, 3), padding='same'))
#     model.add(GlobalAveragePooling2D())
#     model.add(Activation('softmax', name='predictions'))
#     return model

In [11]:
model = create_model()
#model = create_model2()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 44, 44, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 43, 43, 128)       32896     
_________________________________________________________________
average_pooling2d_1 (Average (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)    

## Create Model json

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [13]:
!mkdir 'trained_models'

mkdir: cannot create directory ‘trained_models’: File exists


In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
patience =12

#log_file_path = base_path + dataset_name + '_emotion_training.log'
#csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = 'trained_models3'
model_names = trained_models_path + '.{epoch:02d}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [15]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

model_info = model.fit_generator(
            train_generator,
            
            steps_per_epoch=950, #len(dataset)/batch_size
            epochs=num_epoch,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=200,
            workers=4,
            shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
950/950 [==============================] - 114s 120ms/step - loss: 1.8368 - acc: 0.2814 - val_loss: 1.7731 - val_acc: 0.3168

Epoch 00001: val_loss improved from inf to 1.77313, saving model to trained_models3.01.hdf5
Epoch 2/60
950/950 [==============================] - 110s 115ms/step - loss: 1.7327 - acc: 0.3379 - val_loss: 1.6957 - val_acc: 0.3493

Epoch 00002: val_loss improved from 1.77313 to 1.69567, saving model to trained_models3.02.hdf5
Epoch 3/60
950/950 [==============================] - 111s 117ms/step - loss: 1.6651 - acc: 0.3685 - val_loss: 1.6347 - val_acc: 0.3774

Epoch 00003: val_loss improved from 1.69567 to 1.63468, saving model to trained_models3.03.hdf5
Epoch 4/60
950/950 [==============================] - 111s 117ms/step - loss: 1.6194 - acc: 0.3899 - val_loss: 1.5846 - val_acc: 0.3940

Epoch 00004: val_loss improved from 1.63468 to 1.58465, saving model to trained_models3.04.hdf5
Epoch 5/60
950/950 [====

## Bottleneck Features

In [0]:
# from keras.applications.resnet50 import ResNet50
# datagen = ImageDataGenerator() 
# train_generator = datagen.flow_from_directory(training_directory, target_size=(48,48),shuffle=False,class_mode='categorical' ,batch_size=9)

#     # get bottleneck features
#     # use pre-trained model and exclude top layer - which is used for classification
# pretrained_model = ResNet50(include_top=False, weights='imagenet')

# #InceptionV3(include_top=False, weights='imagenet', input_shape=(48,48,1))
# bottleneck_features_train_v1 = pretrained_model.predict_generator(train_generator,len(train_generator.filenames)//9)

In [0]:
# import numpy as np
# np.save('bottleneck_features_train.npy', bottleneck_features_train_v1)
# # bottleneck_features_train_v1

In [0]:
val_generator = datagen.flow_from_directory(
        validation_directory,
        target_size=(48, 48),
        batch_size=15,
        class_mode='categorical',
        shuffle=False)
bottleneck_features_validation = pretrained_model.predict_generator(val_generator,len(val_generator.filenames)//15)

In [0]:
np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

In [0]:
train_data = np.load('bottleneck_features_train.npy')
validation_data = np.load('bottleneck_features_validation.npy')
num_classes = len(train_generator.class_indices)

In [0]:
datagen_top = ImageDataGenerator(rescale=1./255)  
generator_top = datagen_top.flow_from_directory(  
         training_directory,  
         target_size=(48, 48),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
   
train_labels = generator_top.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [0]:
len(train_labels)

In [0]:
generator_top = datagen_top.flow_from_directory(  
         validation_directory,  
         target_size=(48, 48),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
      
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)  

In [0]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = train_data.shape[1:]))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(output_dim = num_classes, activation='softmax'))

In [0]:
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

model.fit(train_data,train_labels,
          epochs=50,
          batch_size = 16,
          validation_data = (validation_data,validation_labels),
          callbacks = callbacks)